In [1]:
# Test AWS Bedrock Claude Sonnet 4 Access
import boto3
import json
from pydantic import BaseModel
from typing import List

import dotenv
dotenv.load_dotenv()

bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

response = bedrock.converse(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    messages=[
        {
            "role": "user", 
            "content": [
                {"text": "Hello! Can you respond with exactly 3 words?"}
            ]
        }
    ]
)

response_text = response['output']['message']['content'][0]['text']
print(f"✅ Claude response: {response_text}")
print(f"Full response structure: {json.dumps(response, indent=2, default=str)}")

✅ Claude response: Yes, I can!
Full response structure: {
  "ResponseMetadata": {
    "RequestId": "5611c473-50a8-45ba-a351-5f429849c37c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 01 Jul 2025 13:18:10 GMT",
      "content-type": "application/json",
      "content-length": "302",
      "connection": "keep-alive",
      "x-amzn-requestid": "5611c473-50a8-45ba-a351-5f429849c37c"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "Yes, I can!"
        }
      ]
    }
  },
  "stopReason": "end_turn",
  "usage": {
    "inputTokens": 19,
    "outputTokens": 8,
    "totalTokens": 27,
    "cacheReadInputTokens": 0,
    "cacheWriteInputTokens": 0
  },
  "metrics": {
    "latencyMs": 1047
  }
}


In [3]:
try:
    import instructor
    from instructor import Mode
    
    # Create Instructor client (same as your constants.py)
    bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
    
    instructor_client = instructor.from_bedrock(
        client=bedrock_client,
        mode=Mode.BEDROCK_TOOLS,
        temperature=0.2,
    )
    
    # Define a test schema (similar to your vocab tools)
    class WordAnalysis(BaseModel):
        word: str
        language: str
        part_of_speech: str
        definition: str
    
    # Test structured output
    result = instructor_client.create(
        model="us.anthropic.claude-sonnet-4-20250514-v1:0",
        response_model=WordAnalysis,
        messages=[
            {"role": "system", "content": "You are a linguistic specialist."},
            {"role": "user", "content": "Analyze the Spanish word 'casa' and provide structured output."}
        ]
    )
    
    print("✅ Instructor integration works!")
    print(f"Structured result: {result}")
    print(f"Type: {type(result)}")
    
except Exception as e:
    print(f"❌ Instructor test failed: {e}")
    print("Make sure you have instructor[bedrock] installed.")


❌ Instructor test failed: An error occurred (ThrottlingException) when calling the Converse operation (reached max retries: 4): Too many requests, please wait before trying again.
Make sure you have instructor[bedrock] installed.


In [ ]:
# Test 3: Test your actual vocab processor integration
print("\n🔍 Testing your vocab processor setup...")

try:
    # Import your constants to test the actual integration
    import sys
    sys.path.append('/Users/andreaspangerl/Desktop/courses/wordweave/agent')
    
    from vocab_processor.constants import LLMVariant, get_llm_client
    from vocab_processor.tools.base_tool import create_llm_response, SystemMessages
    
    # Test the actual enum and function you built
    claude_client = get_llm_client(LLMVariant.CLAUDE4S)
    print(f"✅ LLMVariant.CLAUDE4S client loaded: {type(claude_client)}")
    
    # Test with your actual create_llm_response function
    class TestSchema(BaseModel):
        spanish_word: str
        english_translation: str
        example_sentence: str
    
    result = await create_llm_response(
        response_model=TestSchema,
        user_prompt="Translate 'perro' to English and give an example sentence in Spanish.",
        system_message=SystemMessages.LINGUISTIC_SPECIALIST,
        llm_provider=LLMVariant.CLAUDE4S
    )
    
    print("✅ Your vocab processor Claude integration works!")
    print(f"Result: {result}")
    
except Exception as e:
    print(f"❌ Vocab processor test failed: {e}")
    print("This might be expected if running outside your project environment.")


In [1]:
from instructor import from_openai, Mode
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv
import os
os.environ["OPENAI_LOG"] = "debug"   # must be set before the first client is built

from vocab_processor.schemas.spanish_conj_model import SpanishVerbConjugation

load_dotenv()

client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"), max_retries=3)
instructor_llm = from_openai(client=client, model="gpt-4o-mini", temperature=0.2, mode=Mode.JSON)



# Define hook functions
def log_kwargs(**kwargs):
    print(f"Function called with kwargs: {kwargs}")


def log_exception(exception: Exception):
    print(f"An exception occurred: {str(exception)}")

    
instructor_llm.on("completion:kwargs", log_kwargs)
instructor_llm.on("completion:error", log_exception)

# x = await instructor_llm.create(
#     messages=[
#         {"role": "system", "content": "You are an expert Spanish linguist. "
#                     "Return ONLY valid JSON matching the schema and do it fast."},
#         {"role": "user", "content": f"Conjugate the verb 'comprar'."}
#     ],
#     response_model=SpanishVerbConjugation,
# )

In [ ]:
import json
print(json.dumps(x.model_dump(), indent=4))

In [ ]:
import dotenv
from vocab_processor.tools.media_tool import fetch_photos
from vocab_processor.schemas.visual_model import Media
load_dotenv()
from pydantic import BaseModel
from pydantic import Field

def get_media(source_word: str, target_word: str, source_language: str, target_language: str) -> Media:
    """
    Get the most memorable photo for a vocabulary word.
    Steps:
    1. Translate word to English with descriptors
    2. Fetch Pexels photos
    3. Ask LLM to choose the best one
    """

    class Criteria(BaseModel):
        query: str = Field(description="English search query plus two descriptive synonyms to find the most relevant photos in Pexels")

    try:
        # translate to english and get criteria
        translation_and_choice_prompt = f"""
        Given the {source_language} word '{source_word}', respond with:
        1. `query` – an English search query plus a few descriptive synonyms to find the most relevant photos in Pexels.

        Respond in the JSON format provided.
        """
        result = instructor_llm.create(response_model=Criteria, messages=[{"role": "user", "content": translation_and_choice_prompt}])


        # fetch three photos
        photos = fetch_photos(result.query, per_page=15)

        print(photos)


        # choose the best photo
        rank_prompt = f"""Find the most adquate photo for the following criteria that depicts best the {source_language} word '{source_word}' that makes it easy to remember the {target_language} word '{target_word}'.

        Photos:
        {[p.model_dump() for p in photos]}

        Respond only in the JSON format provided.
        """

        print(rank_prompt)

        result = instructor_llm.create(
            response_model=Media,
            messages=[{"role": "user", "content": rank_prompt}],
        )

        print(result)
            
    except Exception as e:
        return f"Error generating visual media: {e!s}"

get_media(source_word="Kaputze", target_word="gorra", source_language="Deutsch", target_language="Spanish")

In [8]:
# from my_agent.tools.pronounciation_tool import LANGUAGE_VOICES
from elevenlabs import VoiceSettings, play, save
from dotenv import load_dotenv
load_dotenv()

from elevenlabs.client import ElevenLabs

LANGUAGE_VOICES = {
    "English": "EXAVITQu4vr4xnSDxMaL",  # Bella - English
    "Spanish": "94zOad0g7T7K4oa7zhDq",  # Mauricio
    "German": "XrExE9yKIg1WjnnlVkGX",   # Matilda - Multi-language
}

target_language = "Spanish"
voice_id = LANGUAGE_VOICES.get(target_language, LANGUAGE_VOICES["English"])
        
client = ElevenLabs()

audio = client.text_to_speech.convert(
    text="Hello",
    voice_id=voice_id,
    language_code="es",
    model_id="eleven_flash_v2_5",
    output_format="mp3_44100_128",
    voice_settings=VoiceSettings(
        speed=0.7,
        stability=0.3,
        similarity_boost=0.2,
        style=0.2,
        use_speaker_boost=True
    )
)

# play(audio=audio)
save(audio, "quisquilloso.mp3")